In [21]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [22]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [23]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [25]:
#ABILoading().get_abi_by_filename(abi_path)

In [5]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,837,691 tx:0xd79754317a6dc99931b457377743e126bfc68b06fcabf3482028c0de9380767f
Swap at block:60,837,697 tx:0xa8225f0ec90fffabfd023736aa96359a965b08357eaaba8db2be011cf43034a5
Swap at block:60,837,700 tx:0x6f0da99f786825c95847590d2535d47bbbf17c5d7321f6b06eac51550245a03e
.


In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xd79754317a6dc99931b457377743e126bfc68b06fcab...,60837691,1724194664,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv2pair,swap,sushi,0xc84f479bf220e38ba3bd0262049bad47aaa673ee,0xa8225f0ec90fffabfd023736aa96359a965b08357eaa...,60837697,1724194678,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv2pair,swap,sushi,0x019011032a7ac3a87ee885b6c08467ac46ad11cd,0x6f0da99f786825c95847590d2535d47bbbf17c5d7321...,60837700,1724194684,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0x882df4b0fb50a229c3b4124eb18c759911485bfb',
  'tx_hash': '0xd79754317a6dc99931b457377743e126bfc68b06fcabf3482028c0de9380767f',
  'blk_num': 60837691,
  'timestamp': 1724194664,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
   'token1': '0x07Ff4e06865de4934409Aa6eCea503b08Cc1C78d',
   'amount0In': 4653060900965215000,
   'amount1Out': 148515235}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0xc84f479bf220e38ba3bd0262049bad47aaa673ee',
  'tx_hash': '0xa8225f0ec90fffabfd023736aa96359a965b08357eaaba8db2be011cf43034a5',
  'blk_num': 60837697,
  'timestamp': 1724194678,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xAcc8e414ceeCF0BBF438f6c4B7417Ca59dcF7E47',
   'token1': '0xAcc8e414ceeCF0BBF438f6c4B7417Ca59dcF7E47',
   'amoun